In [1]:
require 'sequel'
DB = Sequel.connect(adapter: 'postgres', 
                    host: 'postgres', 
                    user: 'postgres',
                    password: 'mysecretpassword',
                    database: 'adr',
                    search_path: ['mpd', 'meddra','reports'])
load 'etl_prr/source_model.rb'
load 'etl_prr/extensions.rb'
load 'etl_prr/transformations.rb'

true

## Target Model

In [2]:
TB = Sequel.connect(adapter: 'postgres', 
                    host: 'postgres', 
                    user: 'postgres',
                    password: 'mysecretpassword',
                    database: 'prr',
                    )

#<Sequel::Postgres::Database: {:adapter=>"postgres", :host=>"postgres", :user=>"postgres", :password=>"mysecretpassword", :database=>"prr"}>

## Relational layer

In [3]:
def create_treports_table!(db_connection)
  db_connection.create_table!(:treports) do
    primary_key :id
    Integer :source_id, unique: true
    Date :vt
  end  
end 

def create_tdrugs_table!(db_connection)
  db_connection.create_table!(:tdrugs) do
    primary_key :id
    Integer :treport_id
    String  :name
    String  :brand_name
    String  :atc
  end  
end 

def create_tadrs_table!(db_connection)
  db_connection.create_table!(:tadrs) do
    primary_key :id
    Integer :treport_id
    String :name
    String :preferred_name
    String :soc
  end  
end  

create_treports_table!(TB)
create_tdrugs_table!(TB)
create_tadrs_table!(TB)


## Model layer

In [4]:
Object.send(:remove_const, :Tdrug) if defined?(Tdrug)
Object.send(:remove_const, :Treport) if defined?(Treport)
Object.send(:remove_const, :Tadr) if defined?(Tadr)

class Treport < Sequel::Model(TB)
  one_to_many :tdrugs
  one_to_many :tadrs 
  one_to_one :source, class: Report, primary_key: :source_id, key: :id 
end 

class Tadr < Sequel::Model(TB)
  one_to_one :treport, primary_key: :treport_id, key: :id

  def tdrugs
    treport.tdrugs
  end
    
end 

class Tdrug < Sequel::Model(TB)
  one_to_one :treport, primary_key: :treport_id, key: :id
  
  def tadrs
    treport.tadrs
  end
  
end 

:tadrs

In [5]:
def prr_transform_batch_load
  
  Report.all.select{|report|  
    report.commercial_drug_administrations(has_atc: true).length >= 1 &&
    report.adrs.length >= 1 &&
    !report.fill_date.nil?
  }.each do |report| 
    
    treport = Treport.create(vt: report.fill_date, source_id: report.id) # create treport
    
    report.commercial_drug_administrations.each do |da| # create treport.tdrugs
      treport.add_tdrug(
        name: da.commercial_drug.drug.name, 
        brand_name: da.commercial_drug.name , 
        atc: da.commercial_drug.atc_code)
    end
    
    report.adrs.each do |adr| # create treport.tadrs
      treport.add_tadr(
      name: adr.name_en, 
      preferred_name: adr.adr_preferred_term.name_en , 
      soc: adr.adr_preferred_term.soc.name_en)
    end 
    
  end  
  
end  

:prr_transform_batch_load

In [6]:
def prr_update_data
  
  sources = Treport.all.to_a.map{|treport| treport.source}
  
  Report.all.each do |report| 
    
    if (report.commercial_drug_administrations(has_atc: true).length >= 1 &&
        report.adrs.length >= 1 &&
        !report.fill_date.nil?  &&
        !sources.include?(report)) then
    
        puts "INSERTED report #{report.id}"  
          
        treport = Treport.create(vt: report.fill_date)
        treport.source_id = report.id
        treport.save  
        
        report.commercial_drug_administrations.each do |da| # create treport.tdrugs
          treport.add_tdrug(
            name: da.commercial_drug.drug.name, 
            brand_name: da.commercial_drug.name , 
            atc: da.commercial_drug.atc_code)
        end
        
        report.adrs.each do |adr| # create treport.tadrs
          treport.add_tadr(
          name: adr.name_en, 
          preferred_name: adr.adr_preferred_term.name_en , 
          soc: adr.adr_preferred_term.soc.name_en)
        end 
        
    end
    
  end  
  
end

:prr_update_data

In [ ]:
prr_update_data

INSERTED report 214813
INSERTED report 44434
INSERTED report 332173
INSERTED report 248516
INSERTED report 344293
INSERTED report 258213
INSERTED report 35113
INSERTED report 346813
INSERTED report 347604
INSERTED report 35172
INSERTED report 27992
INSERTED report 74797
INSERTED report 28029
INSERTED report 28055
INSERTED report 262329
INSERTED report 358425
INSERTED report 28180
INSERTED report 358763
INSERTED report 44928
INSERTED report 28362
INSERTED report 365493
INSERTED report 362073
INSERTED report 28421
INSERTED report 28427
INSERTED report 248845
INSERTED report 28513
INSERTED report 368640
INSERTED report 373012
INSERTED report 63321
INSERTED report 373592
INSERTED report 249021
INSERTED report 374512
INSERTED report 376190
INSERTED report 45474
INSERTED report 375371
INSERTED report 381452
INSERTED report 352145
INSERTED report 386152
INSERTED report 40414
INSERTED report 258690
INSERTED report 276707
INSERTED report 382550
INSERTED report 385625
INSERTED report 249292
INSE

## Extraction Transformation and Loading (test)

In [ ]:
report = Report.all.find{|report|  
  report.commercial_drug_administrations.length > 1 &&
  report.adrs.length > 1 &&
  !report.fill_date.nil?
}; nil

In [ ]:
treport = Treport.create(vt: report.fill_date); nil

In [ ]:
report.commercial_drug_administrations.each do |da|
  treport.add_tdrug(
    name: da.commercial_drug.drug.name, 
    brand_name: da.commercial_drug.name , 
    atc: da.commercial_drug.atc_code)
end ; nil 

In [ ]:
treport.tdrugs

In [ ]:
report.adrs.each do |adr|
  treport.add_tadr(
    name: adr.name_en, 
    preferred_name: adr.adr_preferred_term.name_en , 
    soc: adr.adr_preferred_term.soc.name_en)
end  

In [ ]:
treport.tadrs

In [ ]:
CommercialDrug.all.find{|d| d.drug.nil?}